# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [2]:

from __future__ import print_function
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [38]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)
# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

-1.7222030810151945
nan
0.9883709786090357
-0.011697167288802613
-1.0960906600993812
nan
1.8573651665815736
0.6191589063245275
1.4228305791135807
0.3526482530692358
-0.3330769083652695
nan
7.652311171386026
2.0350077161178977
-2.064811007801427
nan
1.9340504104558123
0.6596164620770383
1.3221072705925625
0.27922688079461366
-3.7558715929892625
nan
1.0125198658481975
0.012442140396888256
0.8760770713145117
-0.13230121096320366
3.889412935665676
1.3582582299600299
-0.7045126794273475
nan
0.6800104182838118
-0.3856471599237445
-144.6443243559408
nan
43.64834643183869
3.7761653991359503
2.2419506541069896
0.807346314642803
4.052999338352684
1.3994571843973755
-10.659817805927158
nan
-1.1350911872301042
nan
-3.2708660675754855
nan
-3.0596080795955896
nan
1.3037382366722103
0.265235704605437
4.393838668488303
1.4802032569679897
-0.5165002003663722
nan
2.0597835025654305
0.7226008814365107
1.18397012117124
0.16887330064612982
3.579107712540118
1.2751135270393534
-4.3893834319449
nan
-0.765674

C:\Users\romam\Desktop\Новая папка (3)\DeepLearning-cs231n\assigments1\cs231n\classifiers\softmax.py:46: RuntimeWarning: invalid value encountered in log
  for j in range(num_classes):
C:\Users\romam\Desktop\Новая папка (3)\DeepLearning-cs231n\assigments1\cs231n\classifiers\softmax.py:47: RuntimeWarning: invalid value encountered in log
  if j == y[i]:



1.4211775789794174
-0.259480225276959
nan
1.2384130184657245
0.2138307361250454
-0.3927528363568985
nan
-0.9543384307048826
nan
-0.5755113824556435
nan
6.459305272323658
1.8655217690273953
1.9592948904033909
0.6725846587227565
1.5541245553886298
0.4409124002062328
-1.7439055907852168
nan
1.4164614073499409
0.34816179484134735
0.9040340768106087
-0.10088822371346486
0.8797167217387839
-0.12815533044812036
-0.49718938534481383
nan
-0.23397329080431578
nan
1.4526772544101698
0.3734082363157129
-7.443078140548794
nan
8.384917218782828
2.126434522690103
-0.5750977367752674
nan
0.9983921001439295
-0.0016091939143741683
2.2336204807732
0.8036238029444865
-1.5643805064480056
nan
1.2575609223238453
0.22917406899425083
2.6630531686984784
0.9794732723472702
0.6563328439567264
-0.4210872346334514
1.484125976758895
0.394826031141492
-0.5846029209328835
nan
1.2391420148170638
0.21441921659275193
-1.07942509838508
nan
2.789655761589997
1.0259182052783196
-1.0953654548904985
nan
1.044128858155855
0.0

-1.0161307490299702
nan
3.30274045860119
1.1947525658466187
-1.1871991729222373
nan
0.89110862695124
-0.11528894317226043
-0.4057631032184807
nan
-0.708293397535641
nan
-445.6417053171944
nan
-1.2420945152777196
nan
-4.258040483020622
nan
-1.3403019813733472
nan
1.146409104150214
0.13663453901303915
-2.145088214076421
nan
233.07174208603539
5.451346312121042
0.7579469681653427
-0.27714185862983504
-0.41784394408267306
nan
6.334388557145653
1.8459932909058678
-0.9114445138620066
nan
-1.6405367763246126
nan
-3.9888861974363206
nan
-12.224554589104958
nan
4.652274487832549
1.5373562371789655
3.536849995529651
1.263236499239094
1.3109006421667115
0.27071441408855323
1.6785896352577063
0.5179539380125373
-0.8058578155950292
nan
-2.1811178177388144
nan
-20.00162575331852
nan
1.7385517666447088
0.5530524486371948
-2.102410546172022
nan
0.6047236590029619
-0.5029836872773994
3.5693720650418546
1.2723896881755714
-0.930311636224797
nan
3.7318370578270907
1.3169006212139005
1.5987924105507252
0.

## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Fill this in*


In [ ]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
# Your code
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question** - *True or False*

It's possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

*Your answer*:

*Your explanation*:

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])